# Demo: Phân Bổ Forecast Tại Nhiều MOB (MOB 12 & 24) + DEL30/DEL90

Use case: Bạn cần forecast tại **2 MOB (12 và 24)** với **DEL30 và DEL90** cho mỗi loan.

In [ ]:
import sys
from pathlib import Path

project_root = Path(".").resolve().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np

from src.config import CFG
from src.data_loader import load_data
from src.rollrate.allocation_multi_mob import (
    allocate_multi_mob_with_del_metrics,
    compare_del_across_mobs,
    export_multi_mob_to_excel,
    pivot_del_by_product_mob,
)

print("✅ Import thành công")

## 1. Load Data

Giả sử bạn đã có:
- `df_raw`: Loan-level data
- `df_lifecycle_final`: Cohort-level forecast (đã calibrated)

In [ ]:
# Load data
# df_raw = load_data("path/to/parquet")
# df_lifecycle_final = ...  # Từ bước calibration trước

print(f"📦 df_raw: {len(df_raw):,} rows")
print(f"📦 df_lifecycle_final: {len(df_lifecycle_final):,} rows")

## 2. Phân Bổ Tại MOB 12 & 24 + Tính DEL30/DEL90

**1 function duy nhất** để:
- Phân bổ forecast tại MOB 12 và MOB 24
- Tính DEL30_FLAG và DEL90_FLAG cho mỗi MOB
- Merge tất cả vào 1 DataFrame

In [ ]:
df_multi_mob = allocate_multi_mob_with_del_metrics(
    df_lifecycle_final=df_lifecycle_final,
    df_raw=df_raw,
    target_mobs=[12, 24],  # 🔥 MOB 12 và 24
    allocation_method="simple",
    include_del30=True,    # 🔥 Tính DEL30
    include_del60=False,
    include_del90=True,    # 🔥 Tính DEL90
)

print(f"\n✅ Kết quả: {len(df_multi_mob):,} loans")
print(f"   Columns: {len(df_multi_mob.columns)}")

df_multi_mob.head()

## 3. Xem Cấu Trúc Output

Output sẽ có format:
```
AGREEMENT_ID | PRODUCT_TYPE | RISK_SCORE | VINTAGE_DATE | MOB_CURRENT | EAD_CURRENT |
STATE_FORECAST_MOB12 | EAD_FORECAST_MOB12 | DEL30_FLAG_MOB12 | DEL90_FLAG_MOB12 |
STATE_FORECAST_MOB24 | EAD_FORECAST_MOB24 | DEL30_FLAG_MOB24 | DEL90_FLAG_MOB24
```

In [ ]:
# Xem columns
print("📋 Columns:")
for col in df_multi_mob.columns:
    print(f"   - {col}")

# Xem sample data
print("\n📊 Sample data:")
df_multi_mob[[
    "AGREEMENT_ID",
    "PRODUCT_TYPE",
    "MOB_CURRENT",
    "STATE_FORECAST_MOB12",
    "DEL30_FLAG_MOB12",
    "DEL90_FLAG_MOB12",
    "STATE_FORECAST_MOB24",
    "DEL30_FLAG_MOB24",
    "DEL90_FLAG_MOB24",
]].head(10)

## 4. Phân Tích: DEL30 Tại MOB 12 vs MOB 24

In [ ]:
# 4.1. Tổng số loans có DEL30=1
print("📊 DEL30 Summary:")
print(f"   MOB 12: {df_multi_mob['DEL30_FLAG_MOB12'].sum():,} loans ({df_multi_mob['DEL30_FLAG_MOB12'].mean()*100:.2f}%)")
print(f"   MOB 24: {df_multi_mob['DEL30_FLAG_MOB24'].sum():,} loans ({df_multi_mob['DEL30_FLAG_MOB24'].mean()*100:.2f}%)")

# 4.2. Migration: 0→1, 1→1, 0→0, 1→0
df_del30_compare = compare_del_across_mobs(
    df_multi_mob=df_multi_mob,
    target_mobs=[12, 24],
    metric="DEL30"
)

df_del30_compare.head()

## 5. Phân Tích: DEL90 Tại MOB 12 vs MOB 24

In [ ]:
# 5.1. Tổng số loans có DEL90=1
print("📊 DEL90 Summary:")
print(f"   MOB 12: {df_multi_mob['DEL90_FLAG_MOB12'].sum():,} loans ({df_multi_mob['DEL90_FLAG_MOB12'].mean()*100:.2f}%)")
print(f"   MOB 24: {df_multi_mob['DEL90_FLAG_MOB24'].sum():,} loans ({df_multi_mob['DEL90_FLAG_MOB24'].mean()*100:.2f}%)")

# 5.2. Migration
df_del90_compare = compare_del_across_mobs(
    df_multi_mob=df_multi_mob,
    target_mobs=[12, 24],
    metric="DEL90"
)

df_del90_compare.head()

## 6. Pivot Table: DEL90 Theo Product × MOB

In [ ]:
# Pivot: DEL90% theo Product và MOB
df_pivot_del90 = pivot_del_by_product_mob(
    df_multi_mob=df_multi_mob,
    target_mobs=[12, 24],
    metric="DEL90"
)

print("📊 DEL90% by Product × MOB:")
print(df_pivot_del90)

# Visualize
import matplotlib.pyplot as plt

df_pivot_del90.plot(kind="bar", figsize=(10, 6))
plt.title("DEL90% by Product (MOB 12 vs MOB 24)")
plt.xlabel("Product")
plt.ylabel("DEL90%")
plt.legend(title="MOB")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 7. Lọc Loans Theo Tiêu Chí

### 7.1. Loans Có DEL90=1 Tại MOB 12

In [ ]:
# Loans dự báo sẽ rơi vào DEL90+ tại MOB 12
high_risk_mob12 = df_multi_mob[
    df_multi_mob["DEL90_FLAG_MOB12"] == 1
].copy()

print(f"⚠️ Có {len(high_risk_mob12):,} loans dự báo DEL90+ tại MOB 12")
print(f"   Tổng EAD: {high_risk_mob12['EAD_FORECAST_MOB12'].sum():,.0f}")

# Top 10 loans có EAD cao nhất
print("\n📋 Top 10 loans có EAD cao nhất:")
top10 = high_risk_mob12.nlargest(10, "EAD_FORECAST_MOB12")
print(top10[[
    "AGREEMENT_ID",
    "PRODUCT_TYPE",
    "STATE_FORECAST_MOB12",
    "EAD_FORECAST_MOB12"
]])

### 7.2. Loans Chuyển Từ DEL30=0 → DEL30=1 (MOB 12 → 24)

In [ ]:
# Loans deteriorate: DEL30=0 tại MOB 12 → DEL30=1 tại MOB 24
deteriorate = df_multi_mob[
    (df_multi_mob["DEL30_FLAG_MOB12"] == 0) &
    (df_multi_mob["DEL30_FLAG_MOB24"] == 1)
].copy()

print(f"📉 Có {len(deteriorate):,} loans deteriorate (0→1) từ MOB 12 đến MOB 24")
print(f"   Tỷ lệ: {len(deteriorate)/len(df_multi_mob)*100:.2f}%")

# Phân tích theo product
print("\n📊 Deterioration by Product:")
print(deteriorate["PRODUCT_TYPE"].value_counts())

### 7.3. Loans Cải Thiện: DEL90=1 → DEL90=0 (MOB 12 → 24)

In [ ]:
# Loans improve: DEL90=1 tại MOB 12 → DEL90=0 tại MOB 24
improve = df_multi_mob[
    (df_multi_mob["DEL90_FLAG_MOB12"] == 1) &
    (df_multi_mob["DEL90_FLAG_MOB24"] == 0)
].copy()

print(f"📈 Có {len(improve):,} loans improve (1→0) từ MOB 12 đến MOB 24")
print(f"   Tỷ lệ: {len(improve)/len(df_multi_mob)*100:.2f}%")

if len(improve) > 0:
    print("\n📋 Sample:")
    print(improve[[
        "AGREEMENT_ID",
        "STATE_FORECAST_MOB12",
        "STATE_FORECAST_MOB24"
    ]].head())

## 8. Export Kết Quả Ra Excel

Export với nhiều sheets:
- All_Loans
- DEL30_MOB12, DEL30_MOB24
- DEL90_MOB12, DEL90_MOB24
- Summary

In [ ]:
export_multi_mob_to_excel(
    df_multi_mob=df_multi_mob,
    filename="outputs/Loan_Forecast_MOB12_MOB24.xlsx",
    target_mobs=[12, 24]
)

print("\n✅ Exported to outputs/Loan_Forecast_MOB12_MOB24.xlsx")

## 9. Visualize: DEL90 Evolution (MOB 12 → 24)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 9.1. Sankey-like visualization (simplified)
migration_counts = df_multi_mob.groupby([
    "DEL90_FLAG_MOB12",
    "DEL90_FLAG_MOB24"
]).size().reset_index(name="count")

migration_counts["label"] = (
    migration_counts["DEL90_FLAG_MOB12"].astype(str) + "→" +
    migration_counts["DEL90_FLAG_MOB24"].astype(str)
)

plt.figure(figsize=(10, 6))
plt.bar(migration_counts["label"], migration_counts["count"])
plt.title("DEL90 Migration (MOB 12 → MOB 24)")
plt.xlabel("Migration")
plt.ylabel("Number of Loans")
plt.tight_layout()
plt.show()

# 9.2. Heatmap: DEL90% by Product × MOB
plt.figure(figsize=(8, 6))
sns.heatmap(df_pivot_del90, annot=True, fmt=".2f", cmap="YlOrRd")
plt.title("DEL90% by Product × MOB")
plt.tight_layout()
plt.show()

## 10. Use Case: IFRS9 ECL Calculation

Tính ECL dựa trên DEL90 tại MOB 12 và MOB 24.

In [ ]:
# Giả sử LGD = 45%, Discount rate = 10%
LGD = 0.45
DISCOUNT_RATE = 0.10

# ECL tại MOB 12 (12-month ECL)
df_multi_mob["ECL_MOB12"] = (
    df_multi_mob["EAD_FORECAST_MOB12"] *
    df_multi_mob["DEL90_FLAG_MOB12"] *
    LGD /
    ((1 + DISCOUNT_RATE) ** (12/12))  # Discount 12 tháng
)

# ECL tại MOB 24 (24-month ECL)
df_multi_mob["ECL_MOB24"] = (
    df_multi_mob["EAD_FORECAST_MOB24"] *
    df_multi_mob["DEL90_FLAG_MOB24"] *
    LGD /
    ((1 + DISCOUNT_RATE) ** (24/12))  # Discount 24 tháng
)

# Tổng ECL
print("📊 ECL Summary:")
print(f"   Total ECL (MOB 12): {df_multi_mob['ECL_MOB12'].sum():,.0f}")
print(f"   Total ECL (MOB 24): {df_multi_mob['ECL_MOB24'].sum():,.0f}")

# ECL theo product
print("\n📊 ECL by Product:")
ecl_by_product = df_multi_mob.groupby("PRODUCT_TYPE")[["ECL_MOB12", "ECL_MOB24"]].sum()
print(ecl_by_product)